# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [1]:
import os
import json
from pathlib import Path
from typing import NamedTuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tqdm

import cv2
import pyrealsense2 as rs

sns.set_style()

In [146]:
RECORDING_DIR = Path('H:/Recordings/')
FIGURES_DIR = Path('../docs/Thesis/figures/')

## Data Loading

Firstly we need to import all the recordings into the notebook.


### Load Metadata

It is important to load the metadata, such as the session parameters, the exercises and the recording paths.

In [3]:
recording_jsons = []
for file in os.listdir(RECORDING_DIR):
  if (file.endswith('.json')):
    with open(file=os.path.join(RECORDING_DIR, file), mode='r') as file:
      data = json.load(file)
      recording_jsons.append(data)

len(recording_jsons)

25

Then we load the Exercises from the exercise file.

In [4]:
with open(file="Exercises.json", mode='r') as file:
  exercises_json = json.load(file)['Exercises']

with open(file="JointErrors.json", mode='r') as file:
  joint_error_json = json.load(file)

with open(file="SkeletonErrors.json", mode='r') as file:
  skeleton_error_json = json.load(file)

len(exercises_json)

13

## Load Frame

Here we define the load frame functions. For now we focus on nuitrack recordings.

In [5]:
def id_2_name(i: int):
  return 'frame_' + str(i) + '.yml'

In [7]:
from dataclasses import dataclass

@dataclass
class Frame:
    rgb: np.ndarray
    depth: np.ndarray
    poses_2d: np.ndarray
    poses_3d: np.ndarray
    errors: np.ndarray

@dataclass
class AugmentationParams:
    flip: bool = False
    crop: bool = False
    crop_random: bool = False
    crop_pad: int = 0
    gaussian: bool = False
    seed: int = -1

In [63]:
# TODO: Unsure if this is a problem but bounding boxes will change making crop size different
#       Possible solution: If multiple frames are used to represent a frame, use the same bounding box for all frames
#                          For query frames, use the bounding box of the first frame and return list of list of frames as video
# Question: Should the videos be overlapping in frames?
def load_skeletons(skeletons_json, flip: bool=False) -> (np.ndarray, np.ndarray, list[tuple[float, float, float]], list[tuple[float, float, float]]):
  poses = []
  pose_errors = []
  bounding_boxes_2d = [(np.inf, np.inf, np.inf), (0, 0, 0)]
  bounding_boxes_3d = [(np.inf, np.inf, np.inf), (0, 0, 0)]
  for person in skeletons_json:
    joints = np.ndarray(shape=[0, 6])
    errors = []    
    origin = person['Skeleton'][4]

    for joint in person['Skeleton']:
      if (joint['error'] != 1):
        bounding_boxes_2d[0] = np.minimum(bounding_boxes_2d[0], [joint['u'], joint['v'], joint['d']])
        bounding_boxes_2d[1] = np.maximum(bounding_boxes_2d[1], [joint['u'], joint['v'], joint['d']])
        bounding_boxes_3d[0] = np.minimum(bounding_boxes_3d[0], [joint['x'], joint['y'], joint['z']])
        bounding_boxes_3d[1] = np.maximum(bounding_boxes_3d[1], [joint['x'], joint['y'], joint['z']])
      joints = np.append(joints, [[
        joint['u'] - origin['u'],
        joint['v'] - origin['v'],
        joint['d'] - origin['d'],
        joint['x'] - origin['x'],
        joint['y'] - origin['y'],
        joint['z'] - origin['z']
      ]], axis=0) * (-1 if flip else 1)

      errors.append(1 if person['error'] == 1 else joint['error'])
              
    poses.append(joints)
    pose_errors.append(errors)
  
  return np.asarray(poses), np.asarray(pose_errors), bounding_boxes_2d, bounding_boxes_3d

In [108]:
def load_frames(session: json, start: int, stop: int, params: AugmentationParams = AugmentationParams()) -> list[Frame]:
  frames = []

  if (start > stop):
    start, stop = stop, start
  if (start < 0):
    start = 0
  if (stop > session['Frames']):
    stop = session['Frames'] 
  
  params.seed = np.random.randint(0, 100000)
  
  for frame_id in range(start, stop):
    frames.append(load_frame(session, frame_id, params))
  
  return frames

def load_frame(session: json, frame_id: int, params: AugmentationParams = AugmentationParams()) -> Frame:
  frame_path = RECORDING_DIR /  session['Cameras'][0]['FileName'] / id_2_name(frame_id)
  print(frame_path)
  frame_file = cv2.FileStorage(str(frame_path), cv2.FileStorage_READ)
  frame = np.asarray( frame_file.getNode('frame').mat()[:,:] )
  rgb, depth = np.split(frame, [3], axis=2)

  with open(file=RECORDING_DIR /  session['Skeleton'], mode='r') as file:
    skeleton_json = json.load(file)[frame_id]
  poses, errors, bounding_boxes_2d, bounding_boxes_3d = load_skeletons(skeleton_json, params.flip)
  pose_2d, pose_3d = np.split(poses, 2, axis=2)
    
  if (params.flip):
    rgb = np.flip(rgb, axis=1)
    depth = np.flip(depth, axis=1)

  if params.crop or params.crop_random:
    min_x = max(0, int(np.floor(bounding_boxes_2d[0][1])) - params.crop_pad)
    min_y = max(0, int(np.floor(bounding_boxes_2d[0][0])) - params.crop_pad)

    max_x = min(rgb.shape[1], int(np.ceil(bounding_boxes_2d[1][1])) + params.crop_pad)
    max_y = min(rgb.shape[0], int(np.ceil(bounding_boxes_2d[1][0])) + params.crop_pad)

    if (params.crop_random):
      if (params.seed == -1):
        seed = np.random.randint(0, 100000)
      else:
        seed = params.seed
      np.random.seed(seed)
      min_x = np.random.randint(0, min_x)
      min_y = np.random.randint(0, min_y)
      max_x = np.random.randint(max_x, rgb.shape[1])
      max_y = np.random.randint(max_y, rgb.shape[0])

    rgb = rgb[min_x:max_x, min_y:max_y]
    depth = depth[min_x:max_x, min_y:max_y] 
  
  if (params.gaussian):
    rgb = cv2.GaussianBlur(rgb, (5, 5), 0)
    depth = cv2.GaussianBlur(depth, (5, 5), 0)

  return Frame(rgb, depth, pose_2d, pose_3d, errors)

In [161]:
frame = load_frame(session=recording_jsons[0], frame_id=25, params=AugmentationParams(flip=False, crop=False, crop_random=False, crop_pad=0, gaussian=True))
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
# print("All Correct Joints")
# print(frame.poses_2d[frame.errors==0])
print("All Missing Joints")
print(frame.poses_2d[frame.errors==1])
print("All Wrong Joints")
print(frame.poses_2d[frame.errors==2])

depth_norm = frame.depth / 5
depth_im = np.dstack((depth_norm, depth_norm, depth_norm)) 
im = np.hstack((frame.rgb, depth_im))

cv2.imwrite(str(FIGURES_DIR / 'Model' / 'Augmentation' / 'GaussianBlur.png'), 255*im)
cv2.imshow('depth', im)
cv2.waitKey()
cv2.destroyAllWindows()

H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_25.yml
All Missing Joints
[[-349.36 -304.77 -2.62]
 [-349.36 -304.77 -2.62]
 [-349.36 -304.77 -2.62]
 [-349.36 -304.77 -2.62]
 [-349.36 -304.77 -2.62]]
All Wrong Joints
[[1.26 168.93 0.26]
 [-0.73 172.46 0.25]]


In [36]:
a = AugmentationParams(flip=True, crop=False, crop_random=False, crop_pad=0, gaussian=True)
frames = load_frames(session=recording_jsons[0], start=0, stop=10, params=a)
for frame in frames:
  print(frames[0].rgb.shape == frame.rgb.shape)
  print(frames[0].depth.shape == frame.depth.shape)

H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_0.yml
crop 286 147 423 477
crop_random 112 78 440 610 51096
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_1.yml
crop 286 147 423 477
crop_random 112 78 440 610 51096
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_2.yml
crop 286 147 423 477
crop_random 112 78 440 610 51096
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_3.yml
crop 286 147 422 477
crop_random 112 78 439 610 51096
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_4.yml
crop 286 147 422 477
crop_random 112 78 439 610 51096
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_5.yml
crop 286 147 422 477
crop_random 112 78 439 610 51096
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_6.yml
crop 286 147 422 477
crop_random 112 78 439 610 51096
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_7.yml
crop 286 147 422 477
crop_random 112 78 439 610 51096
H:\Recordings\Session_2023-02-28T22.58.23\Frames\frame_8.yml
crop 287 147 422 47

In [ ]:
poses_2d

array([[[ 3.49257782e+02,  3.04989929e+02,  2.61749840e+00],
        [-1.84570312e-01, -1.57876678e+02, -1.63670063e-01],
        [ 4.53582764e-01,  1.26848038e+02,  1.31840229e-01],
        [ 2.45849609e-01, -4.82750244e+01, -1.82793140e-02],
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [-2.71636963e-01, -1.06412277e+02, -1.03270292e-01],
        [-4.05320740e+01,  1.04062683e+02,  9.98923779e-02],
        [ 5.69803772e+01, -4.21243286e+01, -3.98993492e-03],
        [-6.95727234e+01, -1.72396545e+01,  4.21814919e-02],
        [ 7.21361389e+01,  2.93243713e+01, -4.98199463e-02],
        [ 3.49257782e+02,  3.04989929e+02,  2.61749840e+00],
        [-2.71636963e-01, -1.06412277e+02, -1.03270292e-01],
        [ 4.05550842e+01,  1.02272476e+02,  8.75868797e-02],
        [-5.27978516e+01, -3.88483887e+01,  5.90515137e-03],
        [ 5.99971619e+01, -2.09994202e+01,  2.67663002e-02],
        [-6.14588928e+01,  3.31505737e+01, -3.33003998e-02],
        [ 3.49257782e+02

## Train Model

In the following we define the training function and train a network on the training data.

In [167]:
from RD3D import train
from RD3D.model.rd3d import RD3D

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from RD3D.model.rd3d import RD3D
from RD3D.data import get_loader
from RD3D.utils.func import AvgMeter, clip_gradient
from RD3D.utils.lr_scheduler import get_scheduler
from RD3D.utils.logger import setup_logger


In [ ]:
# training
def train_salient(train_loader, model, optimizer, criterion, scheduler, epoch, opt):
    # multi-scale training  
    size_rates = [0.75, 1, 1.25]

    model.train()
    loss_record = AvgMeter()
    for i, pack in enumerate(train_loader, start=1):
        for rate in size_rates:
            optimizer.zero_grad()
            images, gts, depths = pack
            images = images.cuda()
            gts = gts.cuda()
            depths = depths.cuda()

            # multi-scale training samples
            trainsize = int(round(opt.trainsize * rate / 32) * 32)
            if rate != 1:
                images = F.upsample(images, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
                images = images.unsqueeze(2)
                gts = F.upsample(gts, size=(trainsize, trainsize), mode='bilinear', align_corners=True)

                depths = F.upsample(depths, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
                depths = depths.unsqueeze(2)
                images = torch.cat([images, depths], 2)

            if rate == 1:
                images = images.unsqueeze(2)
                depths = depths.unsqueeze(2)
                images = torch.cat([images, depths], 2)

            # forward
            pred_s = model(images)
            # TODO Calculate different loss based on the error label
            loss = criterion(pred_s, gts)

            loss.backward()
            clip_gradient(optimizer, opt.clip)
            optimizer.step()
            scheduler.step()
            if rate == 1:
                loss_record.update(loss.data, opt.batchsize)

        if i % 100 == 0 or i == len(train_loader):
            logger.info('Epoch [{:03d}/{:03d}], Step [{:04d}/{:04d}], Loss: {:.4f}'.
                        format(epoch, opt.epochs, i, len(train_loader),
                               loss_record.show()))